# Energy efficient parallel programming

Urszula Kicinger, Szymon Ryś (based on exercises from Kamil Jarosz)

**Note:** this notebook is meant to be executed on the supplied Docker image.

**Aim:** The aim of this notebook is to present energy efficient programming methods and relation between execution time and used energy while execution graph algorithms.

In [ ]:
# You may need to additionally install matplotlib

# %pip install matplotlib

In [ ]:
import os
from pathlib import Path
import time

import matplotlib.pyplot as plt

## RAPL sysfs Interface


### CPU zone in sysfs

First of all, it's important to find the zone of your CPU in sysfs. Use the knowledge from the previous modules to find the path of thiz zone and complete the code below.

Please verify your path to the RAPL module in your PC.


In [ ]:
cpu_zone = '/sys/...'

Let's verify whether this zone seems like the zone of your CPU.

In [ ]:
if os.path.isdir(cpu_zone):
    print('✓ Zone exists')
else:
    print('✗ Zone does not exist!')

name = Path(f'{cpu_zone}/name').read_text().strip()

if name.startswith('package-'):
    print('✓ Its name starts with \'package-\'')
else:
    print('✗ Its zone does not start with \'package-\'!')

if os.path.isfile(f'{cpu_zone}/energy_uj'):
    print('✓ File \'energy_uj\' exists')
else:
    print('✗ File \'energy_uj\' does not exist!')

### Energy consumption measurements

The energy counter which reports energy consumed by the zone in micro joules is available as a file named `energy_uj`. This file returns the energy consumed from ar arbitrary, but fixed point in time. By calculating the difference between these values at different points in time, we can obtain the energy consumed during the measurement period.


Let's start with rading the value of `energy_uj`. Complete the following code.

In [ ]:
def energy_uj():
    raise Exception("Unimplemented 'energy_uj'")
    # return 

print(f'Current energy counter value: {energy_uj()}')

Next, we can use the energy counter value read at two different times to calculate energy consumption during the measurement period. Complete the following code. Remembebr that the counter may be reset to zero!


In [ ]:
def energy_consumption(energy_uj_start, energy_uj_end):
    raise Exception("Unimplemented 'energy_consumption'")
    # return 

Let's verify whether the function defined above works. Check the output of the script below.

In [ ]:
energy = energy_consumption(10000, 10000)
if energy == 0:
    print('✓ Pass')
else:
    print(f'✗ Fail: {energy}')

energy = energy_consumption(0, 10000)
if energy == 10000:
    print('✓ Pass')
else:
    print(f'✗ Fail: {energy}')

counter_max = int(Path(f'{cpu_zone}/max_energy_range_uj').read_text().strip())
energy = energy_consumption(counter_max - 1000, 1000)
if energy == 2000:
    print('✓ Pass')
else:
    print(f'✗ Fail: {energy}')

### Energy consumption measurement - example

Let's try measuring idle CPU energy consumption using the functions `energy_uj()` and `energy_consumption()`. Complete the following code to get the energy consumed during the 5 second measurement period in joules.

In [ ]:
measure_time = 5  # seconds

# time.sleep(...)

print(f'Energy consumed: {energy_consumed_j} J')

The energy consumed should be roughly between 10 and 1000 joules depending on the power of your equipment.

**Note**: If you use the ultrabook with low energy consumprion CPU like [Intel® Core™ i7-1165G7](https://ark.intel.com/content/www/us/en/ark/products/208921/intel-core-i7-1165g7-processor-12m-cache-up-to-4-70-ghz-with-ipu.html) the consumed energy may be below 10 joules

### Modifications of power limits

Verify the usage after power capping. Write the function to read power caps and power profile names.

In the following files you should read:
1. `constraint_{i}_power_limit_uw`
2. `constraint_{i}_max_power_uw`
3. `constraint_{i}_time_window_us`
4. `constraint_{i}_name`

**Tip:** your method may be similar to `energy_uj()`<br>
**Note:** you can have more than one profiles


In [ ]:
def read_power_limit(constraint_num: int):
    """ 
        Function should print information about given constraint and return power limit.
            Name: long_term
            Power limit: .. [uw]
            Max limit: ... [uw]
            Time window: ... [us]
    """
    raise Exception("Unimplemented 'read_power_limit'")
    # return power_limit

Verify your implementation:

In [ ]:
init_power_limit_0 = read_power_limit(0)
init_power_limit_1 = read_power_limit(1)

Now change the power limits. Implement the methods below. Note the provided power limits are in µW (microWatts) 1W = 1 000 000 µW 

In [ ]:
def change_power_limits_uw(
    constraint_num: int,
    new_power_val: int,
    verbose: bool = False
):
    """ 
        Function should change power limit of given constraint to a given new value.
    """
    raise Exception("Unimplemented 'change_power_limits_uw'")
    
    if verbose:
        print(f"The constraint_{constraint_num}_power_limit_uw was changed to {new_power_val}\n")
    return
    

Verify if the power limit has changed.

Be very **careful**!\
Remember old values for changed constraint!\
**Do not decrease** the value too much!

In [ ]:
constraint_num = 1
new_power_limit = 50000000


print("Before changing")
read_power_limit(constraint_num=constraint_num)

change_power_limits_uw(constraint_num=constraint_num, new_power_val=new_power_limit, verbose=True)

print("After changing")
read_power_limit(constraint_num=constraint_num)

# Change power limit to initial state
change_power_limits_uw(constraint_num=constraint_num, new_power_val=init_power_limit_1, verbose=True)

### Powercapping - analysis

In [ ]:
def edp(E, T):
    """ 
        Calculate EDP (Energy Delay Product) metric 
    
            E: energy consumption depending on power limit [J]
            T: execution time depending on power limit [s]
    """
    raise Exception("Unimplemented 'edp'")
    # return

def draw_power_plot(E, T, P):
    """ 
        Draw 2 subplots: power(power limits) and EDP(power limits)
        
            E: energy consumption depending on power limit [J]
            T: execution time depending on power limit [s]
            P: power limits [W]
    """

    fig, ax = plt.subplots(1,2, figsize=(12, 6))

    power = ...
    ax[0].scatter(P, power)
    ax[0].plot(P, power, '--')
    ax[0].set_ylabel("Power [W]")
    ax[0].set_xlabel("Power limit [W]")
    ax[0].set_title("Power usage depending on set power limit")

    edp_vals = ...
    ax[1].scatter(P, edp_vals)
    ax[1].plot(P, edp_vals, '--')
    ax[1].set_ylabel("EDP")
    ax[1].set_xlabel("Power limit [W]")
    ax[1].set_title("EDP depending on set power limit")

    plt.show()

def draw_energy_plots(E, T, P):
    """ 
        Draw plots on 1 graph E(power_limit), T(power_limit)
        
            E: energy consumption depending on power limit [J]
            T: execution time depending on power limit [s]
            P: power limits [W]
    """

    fig, ax = plt.subplots()

    fig.suptitle("Power capping - measurements")

    ax1 = ax.twinx()

    ax.scatter(P, E, color='tab:blue')
    ax.plot(P,E, '--', color='tab:blue')
    ax.set_ylabel("Energy [J]", color='tab:blue')
    ax.set_xlabel("Power limit [W]")
    ax.tick_params(axis='y', labelcolor='tab:blue')

    ax1.scatter(P, T, color='tab:green')
    ax1.plot(P,T, '--', color='tab:green')
    ax1.set_ylabel("Time [s]", color='tab:green')
    ax1.set_xlabel("Power limit [W]")
    ax1.tick_params(axis='y', labelcolor='tab:green')

    plt.show()

## Energy efficiency and power consumption measurements based on matrix multiplication

First problem that we are going to measure efficiency is matrix multiplication. In this task we are going to use Numpy library that implements quick matrix multiplication with parallelism.  

In [ ]:
import numpy as np

In [ ]:
def np_test(a, b):
    """ 
        Create a that measures execution time and consumed energy while executing matrix multiplication.
        
        Return: execution_time, energy_consumed
    """"

    raise Exception("Unimplemented 'np_test'")

    # return execution_time, energy_consumed

Define power limits (at least 5 different) that you're going to set and make tests.

In [ ]:
power_limits = ...

Run tests and analyse the results.

In [ ]:
matrix_size = 10000

np.random.seed(42)
test_matrices = (np.random.rand(matrix_size, matrix_size), np.random.rand(matrix_size, matrix_size))

ex_time = np.zeros((n_power_limits, n_tests))
energy = np.zeros((n_power_limits, n_tests))

idx = 0
for limit in power_limits:
    change_power_limits_uw(1, limit)

    t, energy_consumed_j = np_test(test_matrices[i][0], test_matrices[i][1])

    ex_time[idx][ = t
    energy[idx] = energy_consumed_j

    idx += 1

change_power_limits_uw(1,init_power_limit_1)

In [ ]:
draw_energy_plots(energy, ex_time, power_limits_w)

In [ ]:
draw_power_plot(energy, ex_time, power_limits_w)

## Energy efficiency and power consumption measurements based on graph algorithms 


Implement a function that will test energy efficiency of a given bfs and dfs algorithms.

In [ ]:
import networkx as nx
import numpy as np

def test_graph_bfs(graph):
    """ 
        Create a function that measures execution time and consumed energy while executing BFS on a given graph.
        
        Return: execution_time, energy_consumed
    """"

    raise Exception("Unimplemented 'test_graph_bfs'")
    # bfs_result = nx.bfs_tree(graph, source=0)
    
    return execution_time, energy_consumed

def test_graph_dfs(graph):
    """ 
        Create a function that measures execution time and consumed energy while executing DFS on a given graph.
        Return: execution_time, energy_consumed
    """"
    
    raise Exception("Unimplemented 'test_graph_dfs'")
    # dfs_result = nx.dfs_tree(graph, source=0)

    return finish_ts-start_ts, energy_consumed_j

Based on the example code find the the used energy during calculation BFS and DFS.

Use decreasing power limit (at least 5 different limits). Measure times. What are your thoughts? 

In [ ]:
power_limits = ...

### BFS measurements

In [ ]:
ex_time = np.zeros((n_power_limits, 1))
energy = np.zeros((n_power_limits, 1))

G = nx.random_internet_as_graph(int(5e4), seed=42)

idx = 0
for limit in power_limits:
    change_power_limits_uw(1, limit)

    t, energy_consumed_j = test_graph_bfs(G)

    ex_time[idx] = t
    energy[idx] = energy_consumed_j

    idx += 1

change_power_limits_uw(1,init_power_limit_1)

In [ ]:
draw_energy_plots(energy, ex_time, power_limits_w)

In [ ]:
draw_power_plot(energy, ex_time, power_limits_w)

### DFS measurements

In [ ]:
ex_time = np.zeros((n_power_limits, 1))
energy = np.zeros((n_power_limits, 1))

G = nx.random_internet_as_graph(int(5e4), seed=42)

idx = 0
for limit in power_limits:
    change_power_limits_uw(1, limit)

    for i in range(n_tests):

        t, energy_consumed_j = test_graph_dfs(G)

        ex_time[idx] = t
        energy[idx] = energy_consumed_j

    idx += 1

change_power_limits_uw(1,init_power_limit_1)

In [ ]:
draw_energy_plots(energy, ex_time, power_limits_w)

In [ ]:
draw_power_plot(energy, ex_time, power_limits_w)